<span style="font-size: 30px;">Importe de bibliotecas e funções</span>

In [11]:
import folium
import branca.colormap
import numpy as np
import pandas as pd
import requests
import json
import locale
import branca
import pandas as pd
import os
import requests
from zipfile import ZipFile
from io import BytesIO
import dtale


In [12]:
def abrir_zip(link):
    # URL do arquivo no Google Drive
    url = link
    file_id = url.split('/')[-2]

    # Gera o dwn_url para o arquivo zip
    dwn_url = 'https://drive.google.com/uc?export=download&id=' + file_id

    # Faz a requisição para baixar o arquivo ZIP
    response = requests.get(dwn_url)
    # Verifica se a requisição foi bem-sucedida
    response.raise_for_status()

    # Use o BytesIO para abrir o arquivo como um arquivo em memória
    zip_file = BytesIO(response.content)

    # Descompacta o arquivo ZIP
    with ZipFile(zip_file, 'r') as z:
        # Extrai o nome do primeiro arquivo CSV
        csv_name = [name for name in z.namelist() if name.endswith('.csv')][0]
        # Extrai o arquivo CSV para a memória
        with z.open(csv_name) as csv_file:
            # Carrega o arquivo CSV em um DataFrame
            arq_exp = pd.read_csv(csv_file, sep=',', engine='python', decimal='.')

    # Mostra as primeiras linhas do DataFrame
    return arq_exp

In [13]:

#Limpeza da tabela e versão final para análise
def exp_final(arquivo_exp_2022):
    # Leitura dos arquivos de código
    cod_ncm = pd.read_csv('NCM_c.txt', sep=';')
    cod_pais = pd.read_csv('PAIS_c.txt', sep=';')
    cod_uni_ncm = pd.read_csv('NCM_UNIDADE.csv', sep=';')
    cod_via = pd.read_csv('VIA.csv', sep=';')
    cod_urf = pd.read_csv('URF_c.txt', sep=';')
    
    dados_22 = pd.read_csv(arquivo_exp_2022, sep=';')
    limpo_22 = dados_22.drop(columns=['CO_URF', 'KG_LIQUIDO'])
    nomes_colunas = {'CO_MES': 'MES', 'SG_UF_NCM': 'UF'}
    limpo_22.rename(columns=nomes_colunas, inplace=True)
    df22_P = limpo_22.merge(cod_pais, on='CO_PAIS', how='inner')
    df22_Pl = df22_P.drop(columns=['CO_PAIS_ISON3', 'NO_PAIS', 'NO_PAIS_ESP', 'CO_PAIS', 'CO_PAIS_ISOA3'])
    df22_pl = df22_Pl.rename(columns={'NO_PAIS_ING': 'Regiao'})
    df22_PU = df22_pl.merge(cod_via, on='CO_VIA', how='inner')
    df22_PUl = df22_PU.rename(columns={'NO_VIA': 'Via'})
    df22_PV = df22_PUl.drop(columns=['CO_VIA'])
    resultado_3 = df22_PV.merge(cod_uni_ncm, on='CO_UNID', how='inner')
    resultado_31 = resultado_3.drop(columns=['NO_UNID', 'CO_UNID'])
    EXP_22F = resultado_31.rename(columns={'SG_UNID': 'UNIDADE','CO_ANO':'Ano'})
    EXP_22Fv2 = EXP_22F.merge(cod_ncm, on='CO_NCM', how='inner')
    EXP_NCM = EXP_22Fv2.drop(columns=['CO_UNID', 'CO_PPI', 'CO_FAT_AGREG', 'CO_CUCI_ITEM', 'CO_CGCE_N3', 'CO_SIIT', 'CO_ISIC_CLASSE', 'CO_EXP_SUBSET', 'CO_SH6', 'CO_PPE', 'NO_NCM_ESP', 'NO_NCM_ING'])
    EXP_F = EXP_NCM.rename(columns={'NO_NCM_POR': 'CATEGORIA_NCM'})
    return EXP_F


In [14]:
#Calculo FOB
def soma_exp(arquivo_tratado):
    arquivo_tratado['Regiao'] = arquivo_tratado['Regiao'].replace('United States', 'United States')
    arquivo_tratado['Regiao'] = arquivo_tratado['Regiao'].replace('united states', 'United States')
    arquivo_tratado['Regiao'] = arquivo_tratado['Regiao'].replace('French Guyana', 'French Guiana')
    arquivo_tratado['Regiao'] = arquivo_tratado['Regiao'].replace('Serbia', 'Republic of Serbia')
    coluna_selecionada1 = arquivo_tratado['Regiao']
    coluna_selecionada2 = arquivo_tratado['VL_FOB']
    df_novo = pd.concat([coluna_selecionada1, coluna_selecionada2], axis=1)
    df_total_exportacao = df_novo.groupby('Regiao')['VL_FOB'].sum().reset_index()
    FOB = df_total_exportacao.sort_values(by='VL_FOB', ascending=False)
    return FOB


In [15]:
#Calculos dos TOP PRÉ-TRATADOS

def export_3(EXPF):
    resultado = EXPF.groupby(['Regiao', 'CATEGORIA_NCM'])['VL_FOB'].sum().reset_index()
    sorted_df = resultado.sort_values(by=['Regiao', 'VL_FOB'], ascending=[True, False])
    TOP = pd.DataFrame()

# Itere pelos grupos de cada país e selecione os três produtos mais exportados.
    for country, group in sorted_df.groupby('Regiao'):
        top_3 = group.head(3)  # Seleciona os três produtos mais exportados em cada país
        TOP = pd.concat([TOP, top_3])
    return TOP


In [16]:
#DF tratado com os 3 mais exportados por país
def tab_final(TOP,FOB):
    #TOP5= pd.read_csv(TOP)
    grupos = TOP.groupby('Regiao')

    # Crie listas para armazenar os três produtos mais exportados
    top1 = []
    top2 = []
    top3 = []

    # Itere sobre os grupos
    for país, grupo in grupos:
        # Ordene o grupo por 'Valor_Exportação' em ordem decrescente
        grupo = grupo.sort_values(by='VL_FOB', ascending=False)

        # Se o país tiver pelo menos um produto
        if not grupo.empty:
            top1.append([país, grupo.iloc[0]['CATEGORIA_NCM']])

        # Se o país tiver pelo menos dois produtos
        if len(grupo) >= 2:
            top2.append([país, grupo.iloc[1]['CATEGORIA_NCM']])

        # Se o país tiver pelo menos três produtos
        if len(grupo) >= 3:
            top3.append([país, grupo.iloc[2]['CATEGORIA_NCM']])

    # Converta as listas em DataFrames
    top1 = pd.DataFrame(top1, columns=['Regiao', 'top1'])
    top2 = pd.DataFrame(top2, columns=['Regiao', 'top2'])
    top3 = pd.DataFrame(top3, columns=['Regiao', 'top3'])

    TOPT = TOP.merge(top1, on='Regiao', how='inner')
    TOPT = TOPT.drop(columns=['CATEGORIA_NCM','VL_FOB'])
    TOPT2 = TOPT.merge(top2, on='Regiao', how='left')
    TOPT3 = TOPT2.merge(top3, on='Regiao', how='left')
    TOPTF = TOPT3.drop_duplicates(subset=['Regiao'])
    #resultado = pd.read_csv(FOB)
    TOPTF = TOPTF.merge(FOB, on='Regiao',how='inner')
    nome_arquivo_csv = (r"C:\Users\lucas\Downloads\TOPTF.csv")
    TOPTF.to_csv(nome_arquivo_csv, index=False)
    return TOPTF

<span style="font-size: 30px;">Importação das Tabelas auxiliares</span>

In [19]:
cod_ncm =  pd.read_csv('NCM_c.txt', sep=';')
cod_pais = pd.read_csv('PAIS_c.txt',sep = ';')
cod_uni_ncm = pd.read_csv('NCM_UNIDADE.csv',sep = ';')
cod_via = pd.read_csv('VIA.csv', sep=';')
cod_urf = pd.read_csv('URF_c.txt',sep=';')

<span style="font-size: 35px;">Exportações 2022.</span>

In [20]:
EXP_2022F = exp_final("EXP_2022.csv")
EXP_2022F

,Ano,MES,CO_NCM,UF,QT_ESTAT,VL_FOB,Regiao,Via,UNIDADE,CATEGORIA_NCM
0,2022,6,85045000,MG,2,19576,Australia,AEREA,UNID.,Outras bobinas de reatância e de auto-indução
1,2022,1,85045000,MG,1,10090,Australia,AEREA,UNID.,Outras bobinas de reatância e de auto-indução
2,2022,7,85045000,SP,12,204,Australia,AEREA,UNID.,Outras bobinas de reatância e de auto-indução
3,2022,12,85045000,SP,12,367,Australia,AEREA,UNID.,Outras bobinas de reatância e de auto-indução
4,2022,8,85045000,SP,3,47,Australia,AEREA,UNID.,Outras bobinas de reatância e de auto-indução
...,...,...,...,...,...,...,...,...,...,...
1495863,2022,9,27160000,PR,695629,73193022,Argentina,CONDUTO/REDE DE TRANSMISSAO,MKW/H,Energia elétrica
1495864,2022,12,27160000,PR,66324,5112657,Argentina,CONDUTO/REDE DE TRANSMISSAO,MKW/H,Energia elétrica
1495865,2022,11,27160000,PR,658828,70046832,Argentina,CONDUTO/REDE DE TRANSMISSAO,MKW/H,Energia elétrica
1495866,2022,10,27160000,PR,638641,70448507,Argentina,CONDUTO/REDE DE TRANSMISSAO,MKW/H,Energia elétrica


In [21]:

def obter_parte_antes_da_virgula(texto):
    partes = str(texto).split(',', 1)  # Usando split para obter apenas 2 partes
    return partes[0].strip() if len(partes) > 0 else texto

EXP_2022F['CATEGORIA_NCM'] = EXP_2022F['CATEGORIA_NCM'].apply(obter_parte_antes_da_virgula)

EXP_2022F

,Ano,MES,CO_NCM,UF,QT_ESTAT,VL_FOB,Regiao,Via,UNIDADE,CATEGORIA_NCM
0,2022,6,85045000,MG,2,19576,Australia,AEREA,UNID.,Outras bobinas de reatância e de auto-indução
1,2022,1,85045000,MG,1,10090,Australia,AEREA,UNID.,Outras bobinas de reatância e de auto-indução
2,2022,7,85045000,SP,12,204,Australia,AEREA,UNID.,Outras bobinas de reatância e de auto-indução
3,2022,12,85045000,SP,12,367,Australia,AEREA,UNID.,Outras bobinas de reatância e de auto-indução
4,2022,8,85045000,SP,3,47,Australia,AEREA,UNID.,Outras bobinas de reatância e de auto-indução
...,...,...,...,...,...,...,...,...,...,...
1495863,2022,9,27160000,PR,695629,73193022,Argentina,CONDUTO/REDE DE TRANSMISSAO,MKW/H,Energia elétrica
1495864,2022,12,27160000,PR,66324,5112657,Argentina,CONDUTO/REDE DE TRANSMISSAO,MKW/H,Energia elétrica
1495865,2022,11,27160000,PR,658828,70046832,Argentina,CONDUTO/REDE DE TRANSMISSAO,MKW/H,Energia elétrica
1495866,2022,10,27160000,PR,638641,70448507,Argentina,CONDUTO/REDE DE TRANSMISSAO,MKW/H,Energia elétrica


<span style="font-size: 35px;">Somatoria das exportações 2022.</span>

In [22]:
FOB_22= soma_exp(EXP_2022F)
#FOB_22['Regiao'] = FOB_22['Regiao'].str.strip()
#FOB_22['Regiao'] = FOB_22['Regiao'].str.lower()
#FOB_22['Regiao'] = FOB_22['Regiao'].astype('category')
FOB_22['Ano'] = '2022'
FOB_22
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_22.csv")
FOB_22.to_csv(nome_arquivo_csv, index=False)
FOB_22

,Regiao,VL_FOB,Ano
46,China,89427761284,2022
231,United States,37437814354,2022
11,Argentina,15344651930,2022
153,Netherlands,11927910064,2022
204,Spain,9747538076,2022
...,...,...,...
92,Guernsey,1783,2022
31,Bouvet Island,218,2022
161,Northern Marianas,180,2022
33,British Indian Ocean Territory,175,2022


<span style="font-size: 35px;">Tabela final exportações 2022</span>

In [23]:
import pandas as pd
top=export_3(EXP_2022F)
result_22 = tab_final(top,FOB_22)
result_22

,Regiao,top1,top2,top3,VL_FOB,Ano
0,\r\nHeard Island and McDonald Islands,Fuel oil,Couros/peles envernizados ou revestidos,Outros ovos frescos de aves da espécie Gallus ...,303714,2022
1,Afghanistan,Pimenta (do gênero Piper),Outros cremes de leite,Café solúvel,2751417,2022
2,Aland Islands,Mantas,Bobinadoras-esticadoras para acabamento de pap...,Máquinas e aparelhos de impressão,2368,2022
3,Albania,Pedaços e miudezas,Carnes desossadas de bovino,Outros açúcares de cana,90501023,2022
4,Algeria,Outros açúcares de cana,Soja,Milho em grão,1917776167,2022
...,...,...,...,...,...,...
240,Wallis and Futuna,Aparelhos para cozinhar e aquecedores de pratos,Pedras preciosas (exceto diamantes) ou semipre...,Partes de aparelhos para cozinhar,17044,2022
241,Western Sahara,Pedaços e miudezas,NaN,NaN,78812,2022
242,Yemen,Carnes de galos/galinhas,Outros açúcares de cana,Milho em grão,327830114,2022
243,Zambia,Outros tratores,Semeadores-adubadores,Grades de discos,6911309,2022


<span style="font-size: 35px;">Exportações 2021</span>

In [24]:
EXP_2021F= exp_final("EXP_2021.csv")
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\EXP_2021F.csv")
EXP_2021F.to_csv(nome_arquivo_csv, index=False)
EXP_2021F

,Ano,MES,CO_NCM,UF,QT_ESTAT,VL_FOB,Regiao,Via,UNIDADE,CATEGORIA_NCM
0,2021,1,40169990,SP,5353,42235,India,MARITIMA,KGL,"Outras obras de borracha vulcanizada, não endu..."
1,2021,12,40169990,MG,37,1160,India,MARITIMA,KGL,"Outras obras de borracha vulcanizada, não endu..."
2,2021,5,40169990,RJ,0,1,India,MARITIMA,KGL,"Outras obras de borracha vulcanizada, não endu..."
3,2021,2,40169990,PR,196,1144,India,MARITIMA,KGL,"Outras obras de borracha vulcanizada, não endu..."
4,2021,11,40169990,MG,62,1535,India,MARITIMA,KGL,"Outras obras de borracha vulcanizada, não endu..."
...,...,...,...,...,...,...,...,...,...,...
1480290,2021,10,5111000,SP,8399,34101,Paraguay,RODOVIARIA,GRAMA,Sêmen de bovino
1480291,2021,11,5111000,SC,7,21635,Paraguay,RODOVIARIA,GRAMA,Sêmen de bovino
1480292,2021,12,5111000,SC,3,9382,Bolivia,RODOVIARIA,GRAMA,Sêmen de bovino
1480293,2021,5,37029800,CE,1,14,United States,AEREA,M,"Outros filmes sensibilizados, não impressionad..."


<span style="font-size: 35px;">Somatória Exportações 2021</span>

In [25]:
# Chame a função soma_exp com o DataFrame
FOB_21 = soma_exp(EXP_2021F)
FOB_21['Ano'] = '2021'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_21.csv")
FOB_21.to_csv(nome_arquivo_csv, index=False)
FOB_21

,Regiao,VL_FOB,Ano
44,China,87907887856,2021
228,United States,31145209269,2021
10,Argentina,11878463042,2021
151,Netherlands,9316019600,2021
43,Chile,7018693189,2021
...,...,...,...
223,Tuvalu,2101,2021
157,Niue,1605,2021
90,Guernsey,1239,2021
1,Aland Islands,132,2021


<span style="font-size: 35px;">Tabela final Exportações 2021</span>

In [26]:
top=export_3(EXP_2021F)
result_21 = tab_final(top,FOB_21)
result_21
#nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_22.csv")
#FOB_22.to_csv(nome_arquivo_csv, index=False)

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,"Carnes de galos/galinhas, não cortadas em peda...","Pedaços e miudezas, comestíveis de galos/galin...","Pimenta (do gênero Piper), não triturada nem e...",15834475,2021
1,Aland Islands,Outras preparações capilares,NaN,NaN,132,2021
2,Albania,"Pedaços e miudezas, comestíveis de galos/galin...","Carnes desossadas de bovino, congeladas","Outras carnes de suíno, congeladas",59496349,2021
3,Algeria,Outros açúcares de cana,"Soja, mesmo triturada, exceto para semeadura","Milho em grão, exceto para semeadura",1518387137,2021
4,Americana Samoa,"Outros pneumáticos novos, de borracha, do tipo...",Carbonetos de constituição química definida ou...,"Enchidos e produtos semelhantes, de carne, de ...",1840235,2021
...,...,...,...,...,...,...
235,Virgin Islands (USA),"Chapas, barras, perfis, tubos e semelhantes, p...","Móveis de madeira, do tipo utilizado em quarto...","Outras madeiras compensadas, constituídas excl...",1230123,2021
236,Wallis and Futuna,Aparelhos para cozinhar e aquecedores de prato...,"Aparelhos de barbear, não elétricos","Outras obras de borracha vulcanizada, não endu...",9118,2021
237,Yemen,"Carnes de galos/galinhas, não cortadas em peda...","Outros açúcares de cana, beterraba, sacarose q...",Outros açúcares de cana,269456993,2021
238,Zambia,Outros móveis de madeira,Bombas para distribuição de combustíveis ou lu...,"Semeadores-adubadores, de plantio direto",4512714,2021


<span style="font-size: 35px;">Exportações 2020.</span>


In [27]:
EXP_2020F= exp_final("EXP_2021.csv")

<span style="font-size: 35px;">Somatoria das exportações 2020</span>

In [28]:
FOB_20 = soma_exp(EXP_2020F)
FOB_20['Ano'] = '2020'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_20.csv")
FOB_20.to_csv(nome_arquivo_csv, index=False)
FOB_20

,Regiao,VL_FOB,Ano
44,China,87907887856,2020
228,United States,31145209269,2020
10,Argentina,11878463042,2020
151,Netherlands,9316019600,2020
43,Chile,7018693189,2020
...,...,...,...
223,Tuvalu,2101,2020
157,Niue,1605,2020
90,Guernsey,1239,2020
1,Aland Islands,132,2020


<span style="font-size: 35px;">Tabela final Exportações 2020</span>

In [29]:
top=export_3(EXP_2020F)
result_20 = tab_final(top,FOB_20)
result_20

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,"Carnes de galos/galinhas, não cortadas em peda...","Pedaços e miudezas, comestíveis de galos/galin...","Pimenta (do gênero Piper), não triturada nem e...",15834475,2020
1,Aland Islands,Outras preparações capilares,NaN,NaN,132,2020
2,Albania,"Pedaços e miudezas, comestíveis de galos/galin...","Carnes desossadas de bovino, congeladas","Outras carnes de suíno, congeladas",59496349,2020
3,Algeria,Outros açúcares de cana,"Soja, mesmo triturada, exceto para semeadura","Milho em grão, exceto para semeadura",1518387137,2020
4,Americana Samoa,"Outros pneumáticos novos, de borracha, do tipo...",Carbonetos de constituição química definida ou...,"Enchidos e produtos semelhantes, de carne, de ...",1840235,2020
...,...,...,...,...,...,...
235,Virgin Islands (USA),"Chapas, barras, perfis, tubos e semelhantes, p...","Móveis de madeira, do tipo utilizado em quarto...","Outras madeiras compensadas, constituídas excl...",1230123,2020
236,Wallis and Futuna,Aparelhos para cozinhar e aquecedores de prato...,"Aparelhos de barbear, não elétricos","Outras obras de borracha vulcanizada, não endu...",9118,2020
237,Yemen,"Carnes de galos/galinhas, não cortadas em peda...","Outros açúcares de cana, beterraba, sacarose q...",Outros açúcares de cana,269456993,2020
238,Zambia,Outros móveis de madeira,Bombas para distribuição de combustíveis ou lu...,"Semeadores-adubadores, de plantio direto",4512714,2020


<span style="font-size: 35px;">Exportações 2019</span>

In [30]:
EXP_2019F= exp_final("EXP_2019.csv")
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\EXP_2019F.csv")
EXP_2019F.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Somatoria das exportações 2019</span>

In [31]:
FOB_19 = soma_exp(EXP_2019F)
FOB_19['Ano'] = '2019'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_19.csv")
FOB_19.to_csv(nome_arquivo_csv, index=False)
FOB_19

,Regiao,VL_FOB,Ano
44,China,63357523149,2019
228,United States,29715896624,2019
10,Argentina,9791499854,2019
151,Netherlands,7159159102,2019
108,Japan,5431771362,2019
...,...,...,...
49,Cook Islands,1212,2019
31,British Indian Ocean Territory,1166,2019
119,Lesotho,1052,2019
159,Northern Marianas,270,2019


<span style="font-size: 35px;">Tabela final Exportações 2019</span>

In [32]:
top=export_3(EXP_2019F)
result_19 = tab_final(top,FOB_19)
result_19

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,"Carnes de galos/galinhas, não cortadas em peda...","Pedaços e miudezas, comestíveis de galos/galin...","Café solúvel, mesmo descafeinado",32459656,2019
1,Aland Islands,"Camisetas, incluindo as interiores, de malha, ...",NaN,NaN,99,2019
2,Albania,"Pedaços e miudezas, comestíveis de galos/galin...","Carnes desossadas de bovino, congeladas","Outras carnes de suíno, congeladas",58385706,2019
3,Algeria,Outros açúcares de cana,"Óleo de soja, em bruto, mesmo degomado","Milho em grão, exceto para semeadura",1032260192,2019
4,Americana Samoa,"Outros pneumáticos novos, de borracha, do tipo...","Enchidos e produtos semelhantes, de carne, de ...",Medicamentos com outros hormônios polipeptídic...,666670,2019
...,...,...,...,...,...,...
236,Wallis and Futuna,Gasóleo (óleo diesel),"Carnes de galos/galinhas, não cortadas em peda...","Preparações desincrustantes, anticorrosivas ou...",502691,2019
237,Western Sahara,"Pedaços e miudezas, comestíveis de galos/galin...",Outras partes de aparelhos para filtrar ou dep...,NaN,21574,2019
238,Yemen,"Carnes de galos/galinhas, não cortadas em peda...",Outros açúcares de cana,"Outros açúcares de cana, beterraba, sacarose q...",338522817,2019
239,Zambia,"Outros pneumáticos novos, de borracha, do tipo...","Outras carregadoras e pás carregadoras, de car...",Outros móveis de madeira,4444012,2019


<span style="font-size: 35px;">Exportações 2018</span>

In [33]:
EXP_2018F=  exp_final("EXP_2018.csv")

<span style="font-size: 25px;">Somataria das exportações 2018</span>

In [34]:
FOB_18 = soma_exp(EXP_2018F)
FOB_18['Ano'] = '2018'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_18.csv")
FOB_18.to_csv(nome_arquivo_csv, index=False)
FOB_18

,Regiao,VL_FOB,Ano
45,China,63929563241,2018
232,United States,28697187835,2018
10,Argentina,14912622709,2018
152,Netherlands,8229965989,2018
44,Chile,6393547598,2018
...,...,...,...
91,Guernsey,7382,2018
46,Christmas Island,3400,2018
29,Bouvet Island,1028,2018
220,Tokelau,231,2018


<span style="font-size: 35px;">Tabela final exportações 2018</span>

In [35]:
top=export_3(EXP_2018F)
result_18 = tab_final(top,FOB_18)
result_18

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,"Carnes de galos/galinhas, não cortadas em peda...","Pedaços e miudezas, comestíveis de galos/galin...","Milho em grão, exceto para semeadura",16994642,2018
1,Aland Islands,"Outras madeiras compensadas, constituídas excl...","Outros artigos de higiene ou de toucador, de p...",Outros artefatos têxteis confeccionados,47519,2018
2,Albania,"Carnes desossadas de bovino, congeladas","Pedaços e miudezas, comestíveis de galos/galin...","Outras carnes de suíno, congeladas",58757571,2018
3,Algeria,Outros açúcares de cana,"Milho em grão, exceto para semeadura","Óleo de soja, em bruto, mesmo degomado",1007572019,2018
4,Americana Samoa,"Enchidos e produtos semelhantes, de carne, de ...","Pedaços e miudezas, comestíveis de galos/galin...","Carnes de galos/galinhas, não cortadas em peda...",213413,2018
...,...,...,...,...,...,...
240,Wallis and Futuna,Gasóleo (óleo diesel),Aparelhos para cozinhar e aquecedores de prato...,Partes de extintores/aparelhos para pulverizar...,87064,2018
241,Western Sahara,Partes e acessórios de motocicletas (inclusive...,NaN,NaN,2,2018
242,Yemen,"Carnes de galos/galinhas, não cortadas em peda...",Outros açúcares de cana,"Outros açúcares de cana, beterraba, sacarose q...",250791635,2018
243,Zambia,"Outros pneumáticos novos, de borracha, do tipo...",Outros móveis de metal,"Pedaços e miudezas, comestíveis de galos/galin...",7243308,2018


<span style="font-size: 35px;">Exportações 2017</span>

In [36]:
EXP_2017F= exp_final("EXP_2017.csv")
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\EXP_2017F.csv")
EXP_2017F.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Somataria das exportações 2017</span>

In [37]:
FOB_17 = soma_exp(EXP_2017F)
FOB_17['Ano'] = '2017'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_17.csv")
FOB_17.to_csv(nome_arquivo_csv, index=False)
FOB_17

,Regiao,VL_FOB,Ano
42,China,47488449966,2017
222,United States,26872491075,2017
9,Argentina,17618822550,2017
145,Netherlands,7416917751,2017
103,Japan,5261210829,2017
...,...,...,...
47,Cook Islands,1752,2017
85,Guernsey,1133,2017
80,Greenland,355,2017
158,Palau,214,2017


<span style="font-size: 35px;">Tabela final exportações 2017</span>

In [38]:
top=export_3(EXP_2017F)
result_17 = tab_final(top,FOB_17)
result_17

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,"Carnes de galos/galinhas, não cortadas em peda...","Pedaços e miudezas, comestíveis de galos/galin...","Enchidos e produtos semelhantes, de carne, de ...",9276428,2017
1,Aland Islands,"Outros acessórios para tubos, de plásticos","Estampas, gravuras e fotografias","Outros cotovelos, curvas e luvas, roscados, pa...",21179,2017
2,Albania,"Pedaços e miudezas, comestíveis de galos/galin...","Outras carnes de suíno, congeladas","Outros açúcares de cana, beterraba, sacarose q...",44788847,2017
3,Algeria,Outros açúcares de cana,"Óleo de soja, em bruto, mesmo degomado","Milho em grão, exceto para semeadura",1186019580,2017
4,Americana Samoa,Outros açúcares de cana,"Enchidos e produtos semelhantes, de carne, de ...","Ladrilhos e placas (lajes), para pavimentação ...",165034,2017
...,...,...,...,...,...,...
230,Virgin Islands (USA),Fuel oil,"Aparelho transmissor de telefonia celular, par...","Outras madeiras compensadas, constituídas excl...",102037012,2017
231,Wallis and Futuna,Outras preparações capilares,"Preparações para ondulação ou alisamento, perm...",Xampus para os cabelos,29317,2017
232,Yemen,"Outros açúcares de cana, beterraba, sacarose q...",Outros açúcares de cana,"Carnes de galos/galinhas, não cortadas em peda...",403724936,2017
233,Zambia,"Outros pneumáticos novos, de borracha, do tipo...","Contadores de líquidos, peso <= 50kg","Outros pulverizadores, para a agricultura ou h...",9242662,2017


<span style="font-size: 35px;">Exportações 2016</span>

In [39]:
EXP_2016F= exp_final("EXP_2016.csv")

<span style="font-size: 20px;">Somatoria das exportações 2016</span>

In [40]:
FOB_16 = soma_exp(EXP_2016F)
FOB_16['Ano'] = '2016'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_16.csv")
FOB_16.to_csv(nome_arquivo_csv, index=False)
FOB_16

,Regiao,VL_FOB,Ano
42,China,35133314867,2016
223,United States,23155033824,2016
9,Argentina,13417339572,2016
146,Netherlands,6954511332,2016
76,Germany,4860766756,2016
...,...,...,...
85,Guernsey,3258,2016
218,Tuvalu,2338,2016
43,Cocos (Keeling) Islands,2180,2016
1,Aland Islands,1054,2016


<span style="font-size: 35px;">Tabela final exportações 2016</span>

In [41]:
top=export_3(EXP_2016F)
result_16 = tab_final(top,FOB_16)
result_16

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,"Carnes de galos/galinhas, não cortadas em peda...","Pedaços e miudezas, comestíveis de galos/galin...","Enchidos e produtos semelhantes, de carne, de ...",8782814,2016
1,Aland Islands,Outras preparações capilares,Xampus para os cabelos,"Preparações para barbear (antes, durante ou após)",1054,2016
2,Albania,"Outros açúcares de cana, beterraba, sacarose q...","Pedaços e miudezas, comestíveis de galos/galin...","Outras carnes de suíno, congeladas",39335464,2016
3,Algeria,Outros açúcares de cana,"Óleo de soja, em bruto, mesmo degomado","Milho em grão, exceto para semeadura",1063292078,2016
4,Americana Samoa,Outros sucos e extratos vegetais,"Outros ladrilhos, etc, de cerâmica, vidrados, ...",Outras obras de alumínio,52205,2016
...,...,...,...,...,...,...
230,Virgin Islands (USA),Fuel oil,"Outras gasolinas, exceto para aviação","Outros ladrilhos, etc, de cerâmica, vidrados, ...",25454872,2016
231,Wallis and Futuna,Aparelhos para cozinhar e aquecedores de prato...,NaN,NaN,6525,2016
232,Yemen,"Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...",Outros açúcares de cana,384358666,2016
233,Zambia,"Outros pneus novos, para veículos de construçã...",Dumpers para transporte de mercadoria >= 85 to...,"Transformadores de dielétrico líquido, de potê...",8765788,2016


In [42]:
#nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_16.csv")
#FOB_16.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Exportações 2015</span>

In [43]:
EXP_2015F= exp_final("EXP_2015.csv")
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\EXP_2015F.csv")
EXP_2015F.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Somataria das exportações 2015</span>

In [44]:
FOB_15 = soma_exp(EXP_2015F)
FOB_15['Ano'] = '2015'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_15.csv")
FOB_15.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Tabela final exportações 2015</span>

In [45]:
top=export_3(EXP_2015F)
result_15 = tab_final(top,FOB_15)
result_15

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,"Carnes de galos/galinhas, não cortadas em peda...","Enchidos e produtos semelhantes, de carne, de ...",Outros papéis e cartões dos tipos utilizados p...,6706901,2015
1,Albania,"Pedaços e miudezas, comestíveis de galos/galin...","Outros açúcares de cana, beterraba, sacarose q...","Outras carnes de suíno, congeladas",36546211,2015
2,Algeria,Outros açúcares de cana,"Milho em grão, exceto para semeadura","Óleo de soja, em bruto, mesmo degomado",992882604,2015
3,Americana Samoa,"Outros açúcares de cana, beterraba, sacarose q...","Outras máquinas e aparelhos para agricultura, ...",Partes de máquinas e aparelhos para agricultur...,40679,2015
4,Andorra,"Bulhão dourado (bullion doré), em formas bruta...",Joalheria de ouro do capitulo 71 da NCM,NaN,152825,2015
...,...,...,...,...,...,...
228,Virgin Islands (UK),"Aviões e outros veículos aéreos, a turbojato, ...","Outros quadros, etc, com aparelhos interruptor...","Outras carnes de suíno, congeladas",15561494,2015
229,Virgin Islands (USA),"Outros ladrilhos, etc, de cerâmica, vidrados, ...","Outras madeiras compensadas, constituídas excl...",Joalheria de ouro do capitulo 71 da NCM,177698,2015
230,Yemen,Outros açúcares de cana,"Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...",214301925,2015
231,Zambia,"Outros pneus novos, para veículos de construçã...",Semeadores-adubadores,Máquinas e aparelhos para indústria de panific...,5334118,2015


<span style="font-size: 35px;">Exportações 2014</span>

In [46]:
EXP_2014F= exp_final("EXP_2014.csv")

<span style="font-size: 28px;">Somatoria das exportações 2014</span>

In [47]:
FOB_14= soma_exp(EXP_2014F)
FOB_14['Ano'] = '2014'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_14.csv")
FOB_14.to_csv(nome_arquivo_csv, index=False)
FOB_14

,Regiao,VL_FOB,Ano
40,China,40611876675,2014
217,United States,27016702588,2014
8,Argentina,14277231375,2014
142,Netherlands,10984244877,2014
100,Japan,6717455170,2014
...,...,...,...
131,"Midway, Islands",4556,2014
64,Falkland Islands (Malvinas),2225,2014
4,Andorra,1676,2014
163,Pitcairn,516,2014


<span style="font-size: 35px;">Tabela final exportações 2014</span>

In [48]:
top=export_3(EXP_2014F)
result_14 = tab_final(top,FOB_14)
result_14

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,"Carnes de galos/galinhas, não cortadas em peda...","Pedaços e miudezas, comestíveis de galos/galin...","Enchidos e produtos semelhantes, de carne, de ...",11721368,2014
1,Albania,"Outras carnes de suíno, congeladas","Outros açúcares de cana, beterraba, sacarose q...","Carnes desossadas de bovino, congeladas",41175637,2014
2,Algeria,Outros açúcares de cana,"Milho em grão, exceto para semeadura","Carnes desossadas de bovino, frescas ou refrig...",1152271900,2014
3,Americana Samoa,"Outros açúcares de cana, beterraba, sacarose q...",NaN,NaN,25457,2014
4,Andorra,Pedras lapidadas/trabalhadas do capítulo 71 da...,Lentes de contato,NaN,1676,2014
...,...,...,...,...,...,...
223,Virgin Islands (UK),"Outros aviões e outros veículos aéreos, de pes...","Ferro fundido bruto não ligado, que contenha, ...",Outras embarcações para o transporte de mercad...,58935025,2014
224,Virgin Islands (USA),"Outros ladrilhos, etc, de cerâmica, vidrados, ...","Outras madeiras compensadas, constituídas excl...",Outras madeiras tropicais serrada ou fendida l...,182996,2014
225,Yemen,"Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...",Outros açúcares de cana,452010032,2014
226,Zambia,"Outros bulldozers e angledozers, de lagartas",Outros aparelhos para pulverizar fungicidas/in...,Outros móveis de metal,5561210,2014


In [49]:
#nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_14.csv")
#FOB_14.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Exportações 2013</span>

In [50]:
EXP_2013F= exp_final("EXP_2013.csv")
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\EXP_2013F.csv")
EXP_2013F.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 25px;">Somataria das exportações 2013</span>

In [51]:
FOB_13 = soma_exp(EXP_2013F)
FOB_13['Ano'] = '2013'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_13.csv")
FOB_13.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Tabela final exportações 2013</span>

In [52]:
top=export_3(EXP_2013F)
result_13 = tab_final(top,FOB_13)
result_13

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,"Carnes de galos/galinhas, não cortadas em peda...","Enchidos e produtos semelhantes, de carne, de ...",Outras partes de aviões ou de helicópteros,12468229,2013
1,Albania,"Outros açúcares de cana, beterraba, sacarose q...","Outras carnes de suíno, congeladas","Pedaços e miudezas, comestíveis de galos/galin...",53795352,2013
2,Algeria,Outros açúcares de cana,"Milho em grão, exceto para semeadura","Óleo de soja, em bruto, mesmo degomado",1199751306,2013
3,Americana Samoa,"Outros açúcares de cana, beterraba, sacarose q...","Outros ladrilhos, etc, de cerâmica, vidrados, ...",NaN,106593,2013
4,Andorra,Outros calçados sola exterior de couro natural...,Joalheria de ouro do capitulo 71 da NCM,Outros calçados,14649,2013
...,...,...,...,...,...,...
223,Virgin Islands (USA),Outros açúcares de cana,"Outras madeiras compensadas, constituídas excl...","Outros ladrilhos, etc, de cerâmica, vidrados, ...",4757727,2013
224,Wallis and Futuna,Outras válvulas para transmissões óleo-hidrául...,"Motor elétrico de corrente contínua, de potênc...","Carrocerias pata ""dumpers""/tratores, exceto ro...",1386,2013
225,Yemen,"Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...",Outros açúcares de cana,464093877,2013
226,Zambia,"Carregadoras-transportadoras, do tipo das util...",Outros tratores,Semeadores-adubadores,17382357,2013


<span style="font-size: 35px;">Exportações 2012</span>

In [53]:
EXP_2012F= exp_final("EXP_2012.csv")

<span style="font-size: 35px;">Somatoria das exportações 2012</span>

In [54]:
FOB_12 = soma_exp(EXP_2012F)
FOB_12['Ano'] = '2012'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_12.csv")
FOB_12.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Tabela final exportações 2012</span>

In [55]:
top=export_3(EXP_2012F)
result_12 = tab_final(top,FOB_12)
result_12

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,"Carnes de galos/galinhas, não cortadas em peda...","Pedaços e miudezas, comestíveis de galos/galin...","Carnes desossadas de bovino, congeladas",8366059,2012
1,Albania,"Outros açúcares de cana, beterraba, sacarose q...","Pedaços e miudezas, comestíveis de galos/galin...","Carnes desossadas de bovino, congeladas",39526485,2012
2,Algeria,Outros açúcares de cana,"Óleo de soja, em bruto, mesmo degomado","Milho em grão, exceto para semeadura",1169358734,2012
3,Americana Samoa,"Outros ladrilhos, etc, de cerâmica, vidrados, ...","Outros açúcares de cana, beterraba, sacarose q...",NaN,88850,2012
4,Andorra,"Caramelos, confeitos, dropes, pastilhas, e pro...","Outros produtos de confeitaria, sem cacau",Outros chocolates e preparações alimentícias c...,366024,2012
...,...,...,...,...,...,...
227,Wake Island,Figos frescos,NaN,NaN,3191,2012
228,Wallis and Futuna,"Carnes de galos/galinhas, não cortadas em peda...",Outros acumuladores elétricos de chumbo,"Outras chapas, folhas, tiras, fitas, películas...",163005,2012
229,Yemen,"Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...","Pedaços e miudezas, comestíveis de galos/galin...",425577355,2012
230,Zambia,"Carregadoras-transportadoras, do tipo das util...",Outros tratores,"Máquinas horizontais, próprias para empacotame...",12276486,2012


In [56]:
#nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_12.csv")
#FOB_12.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Exportações 2011</span>

In [57]:
EXP_2011F= exp_final("EXP_2011.csv")
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\EXP_2011F.csv")
EXP_2011F.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 20px;">Somataria das exportações 2011</span>

In [58]:
FOB_11= soma_exp(EXP_2011F)
FOB_11['Ano'] = '2011'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_11.csv")
FOB_11.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Tabela final exportações 2011</span>

In [59]:
top=export_3(EXP_2011F)
result_11 = tab_final(top,FOB_11)
result_11

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,"Carnes de galos/galinhas, não cortadas em peda...","Pedaços e miudezas, comestíveis de galos/galin...","Carnes desossadas de bovino, congeladas",10833987,2011
1,Albania,"Outras carnes de suíno, congeladas","Outros açúcares de cana, beterraba, sacarose q...","Carnes desossadas de bovino, congeladas",64254917,2011
2,Algeria,"Açúcar de cana, em bruto","Trigo (exceto trigo duro ou para semeadura), e...","Milho em grão, exceto para semeadura",1493714314,2011
3,Americana Samoa,"Outros ladrilhos, etc, de cerâmica, vidrados, ...","Enchidos e produtos semelhantes, de carne, de ...","Pedaços e miudezas, comestíveis de galos/galin...",113927,2011
4,Andorra,"Caramelos, confeitos, dropes, pastilhas, e pro...","Leite modificado, para alimentação de crianças...","Outros produtos de confeitaria, sem cacau",171243,2011
...,...,...,...,...,...,...
222,Virgin Islands (USA),"Outros ladrilhos, etc, de cerâmica, vidrados, ...",Sacos cuja base tenha largura igual ou superio...,"Móveis de madeira, do tipo utilizado em quarto...",633414,2011
223,Wake Island,"Outras obras de pedras preciosas/semi, sintéti...",NaN,NaN,1660,2011
224,Yemen,"Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...","Trigo (exceto trigo duro ou para semeadura), e...",307646509,2011
225,Zambia,Outros tratores,Semeadores-adubadores,Outras máquinas e aparelhos para colheita,8982609,2011


<span style="font-size: 35px;">Exportações 2010</span>

In [60]:
EXP_2010F= exp_final("EXP_2010.csv")

<span style="font-size: 20px;">Somatoria das exportações 2010</span>

In [61]:
FOB_10 = soma_exp(EXP_2010F)
FOB_19['Ano'] = '2010'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_10.csv")
FOB_10.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Tabela final exportações 2010</span>

In [62]:
top=export_3(EXP_2010F)
result_10 = tab_final(top,FOB_10)
result_10

,Regiao,top1,top2,top3,VL_FOB
0,Afghanistan,"Carnes de galos/galinhas, não cortadas em peda...","Pedaços e miudezas, comestíveis de galos/galin...","Outros açúcares de cana, beterraba, sacarose q...",8350508
1,Albania,"Outras carnes de suíno, congeladas","Carnes de galos/galinhas, não cortadas em peda...","Pedaços e miudezas, comestíveis de galos/galin...",33663381
2,Algeria,"Açúcar de cana, em bruto","Carnes desossadas de bovino, congeladas","Óleo de soja, em bruto, mesmo degomado",838751875
3,Andorra,Pedras lapidadas/trabalhadas do capítulo 71 da...,Joalheria de ouro do capitulo 71 da NCM,Outros chocolates e preparações alimentícias c...,105566
4,Angola,"Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...","Pedaços e miudezas, comestíveis de galos/galin...",944349251
...,...,...,...,...,...
221,Virgin Islands (USA),"Aviões e outros veículos aéreos, a turbojato, ...","Outros ladrilhos, etc, de cerâmica, vidrados, ...",Outros móveis de madeira,3645838
222,Wallis and Futuna,Lentes de contato,Outros calçados sola exterior borracha/plástic...,NaN,1412
223,Yemen,"Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...","Milho em grão, exceto para semeadura",416148118
224,Zambia,Outros tratores,Outras formas de amianto,"Fios de alumínio não ligado, com a maior dimen...",8891157


In [63]:
#nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_10.csv")
#FOB_10.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Exportações 2009</span>

In [64]:
EXP_2009F= exp_final("EXP_2009.csv")
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\EXP_2009F.csv")
EXP_2009F.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 20Px;">Somataria das exportações 2009</span>

In [65]:
FOB_09 = soma_exp(EXP_2009F)
FOB_09['Ano'] = '2009'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_09.csv")
FOB_09.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Tabela final exportações 2009</span>

In [66]:
top=export_3(EXP_2009F)
result_09 = tab_final(top,FOB_09)
result_09

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,"Carnes de galos/galinhas, não cortadas em peda...","Pedaços e miudezas, comestíveis de galos/galin...","Carnes desossadas de bovino, congeladas",8764299,2009
1,Albania,"Outras carnes de suíno, congeladas","Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...",48236544,2009
2,Algeria,"Açúcar de cana, em bruto","Carnes desossadas de bovino, congeladas","Óleo de soja, em bruto, mesmo degomado",712801871,2009
3,Americana Samoa,"Preparações alimentícias e conservas, da espéc...",NaN,NaN,52500,2009
4,Andorra,"Pedaços e miudezas, comestíveis de galos/galin...",Panetone,"Gomas de mascar, mesmo revestidas de açúcar, s...",101037,2009
...,...,...,...,...,...,...
223,Virgin Islands (UK),"Aviões e outros veículos aéreos, a turbojato, ...",Veículos automóveis para transporte de dez pes...,"Outros tubos de ligas de aços, não revestidos,...",71766305,2009
224,Virgin Islands (USA),"Álcool etílico não desnaturado, com volume de ...","Outros ladrilhos, etc, de cerâmica, vidrados, ...",Outros calçados com sola exterior e parte supe...,5318070,2009
225,Yemen,"Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...","Milho em grão, exceto para semeadura",365754002,2009
226,Zambia,Outros tratores,"Outros pneus novos, para veículos de construçã...","Móveis de madeira, do tipo utilizado em cozinhas",5025661,2009


<span style="font-size: 35px;">Exportações 2008</span>

In [67]:
EXP_2008F= exp_final("EXP_2008.csv")

<span style="font-size: 25px;">Somatoria das exportações 2008</span>

In [68]:
FOB_08 = soma_exp(EXP_2008F)
FOB_08['Ano'] = '2008'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_08.csv")
FOB_08.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Tabela final exportações 2008</span>

In [69]:
top=export_3(EXP_2008F)
result_08 = tab_final(top,FOB_08)
result_08

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,"Carnes de galos/galinhas, não cortadas em peda...","Pedaços e miudezas, comestíveis de galos/galin...","Carnes desossadas de bovino, congeladas",7670243,2008
1,Albania,"Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...","Carnes desossadas de bovino, congeladas",50835276,2008
2,Algeria,"Açúcar de cana, em bruto","Carnes desossadas de bovino, congeladas","Outros açúcares de cana, beterraba, sacarose q...",631379893,2008
3,Americana Samoa,"Camisetas, incluindo as interiores, de malha, ...",Bolas infláveis,"Outros chapéus e artefatos de uso semelhante, ...",700,2008
4,Andorra,Outros calçados sola exterior borracha/plástic...,Joalheria de ouro do capitulo 71 da NCM,"Camisas, blusas, blusas chemisiers, de malha, ...",21591,2008
...,...,...,...,...,...,...
222,Virgin Islands (UK),"Aviões e outros veículos aéreos, a turbojato, ...","Outros barcos/embarcações de recreio/esporte, ...","Outros tabaco não manufaturados, total ou parc...",34005291,2008
223,Virgin Islands (USA),"Álcool etílico não desnaturado, com volume de ...","Outros ladrilhos, etc, de cerâmica, vidrados, ...","Artefatos de joalharia, de outros metais preci...",86618760,2008
224,Yemen,"Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...",Milho para semeadura,213996055,2008
225,Zambia,"Fios de alumínio não ligado, com a maior dimen...",Outros tratores,"Carregadoras-transportadoras, do tipo das util...",14227991,2008


In [70]:
#nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_08.csv")
#FOB_08.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Exportações 2007</span>

In [71]:
EXP_2007F= exp_final("EXP_2007.csv")
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\EXP_2007F.csv")
EXP_2007F.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Somataria das exportações 2007</span>

In [72]:
FOB_07 = soma_exp(EXP_2007F)
FOB_07['Ano'] = '2007'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_07.csv")
FOB_07.to_csv(nome_arquivo_csv, index=False)
FOB_07

,Regiao,VL_FOB,Ano
218,United States,25051277380,2007
8,Argentina,14409066223,2007
41,China,10776728025,2007
145,Netherlands,8135147010,2007
76,Germany,7208913806,2007
...,...,...,...
43,Cocos (Keeling) Islands,11144,2007
160,Palau,8824,2007
103,Johnston (Island),5603,2007
138,Montserrat,5582,2007


<span style="font-size: 35px;">Tabela final exportações 2007</span>

In [73]:
top=export_3(EXP_2007F)
result_07 = tab_final(top,FOB_07)
result_07

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,"Carnes de galos/galinhas, não cortadas em peda...","Carnes desossadas de bovino, congeladas","Pedaços e miudezas, comestíveis de galos/galin...",3207691,2007
1,Albania,"Outras carnes de suíno, congeladas","Carnes de galos/galinhas, não cortadas em peda...","Carnes desossadas de bovino, congeladas",44853336,2007
2,Algeria,"Açúcar de cana, em bruto","Carnes desossadas de bovino, congeladas","Leite integral, em pó, com um teor, em peso, d...",500727684,2007
3,Americana Samoa,"Suco (sumo) de laranja, não fermentados, sem a...","Café solúvel, mesmo descafeinado","Caramelos, confeitos, dropes, pastilhas, e pro...",74294,2007
4,Andorra,Outras pedras preciosas (exceto diamantes) ou ...,Outras válvulas cardíacas,Joalheria de ouro do capitulo 71 da NCM,34492,2007
...,...,...,...,...,...,...
226,Western Sahara,"Algodão não cardado nem penteado, simplesmente...","Carnes desossadas de bovino, congeladas",NaN,296327,2007
227,Yemen,"Carnes de galos/galinhas, não cortadas em peda...","Outros açúcares de cana, beterraba, sacarose q...","Produtos laminados planos, de ferro ou aço não...",161947757,2007
228,Yugoslavia,"Pedaços e miudezas, comestíveis de galos/galin...","Carnes de galos/galinhas, não cortadas em peda...",Outros calçados de couro natural,77906,2007
229,Zambia,Outros tratores,"Fios de alumínio não ligado, com a maior dimen...","Leite integral, em pó, com um teor, em peso, d...",11138837,2007


<span style="font-size: 35px;">Exportações 2006</span>

In [74]:
EXP_2006F= exp_final("EXP_2006.csv")

<span style="font-size: 20px;">Somatoria das exportações 2006</span>

In [75]:
FOB_06 = soma_exp(EXP_2006F)
FOB_06['Ano'] = '2006'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_06.csv")
FOB_06.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Tabela final exportações 2006</span>

In [76]:
top=export_3(EXP_2006F)
result_06 = tab_final(top,FOB_06)
result_06

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,"Outros açúcares de cana, beterraba, sacarose q...","Carnes desossadas de bovino, congeladas","Pedaços e miudezas, comestíveis de galos/galin...",2683261,2006
1,Albania,"Outras carnes de suíno, congeladas","Carnes desossadas de bovino, congeladas","Carnes de galos/galinhas, não cortadas em peda...",25611138,2006
2,Algeria,"Açúcar de cana, em bruto","Carnes desossadas de bovino, congeladas","Outros açúcares de cana, beterraba, sacarose q...",455733368,2006
3,Americana Samoa,"Outros bulldozers e angledozers, de lagartas",Outros niveladores,NaN,819226,2006
4,Andorra,Outros uísques,Outras pedras preciosas (exceto diamantes) ou ...,Rum e outras aguardentes provenientes da desti...,27211,2006
...,...,...,...,...,...,...
231,Western Sahara,Ferro-nióbio,"Roupas de toucador ou de cozinha, de tecidos a...",NaN,696138,2006
232,Yemen,"Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...","Açúcar de cana, em bruto",279031812,2006
233,Yugoslavia,"Carnes desossadas de bovino, congeladas","Café não torrado, não descafeinado, em grão","Tabaco não manufaturado, total ou parcialmente...",22460099,2006
234,Zambia,Outros tratores,"Fios de alumínio não ligado, com a maior dimen...",Outras ferramentas de perfuração ou de sondage...,7503549,2006


In [77]:
#nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_06.csv")
#FOB_06.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Exportações 2005</span>

In [78]:
EXP_2005F= exp_final("EXP_2005.csv")
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\EXP_2005F.csv")
EXP_2005F.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 20px;">Somataria das exportações 2005</span>

In [79]:
FOB_05 = soma_exp(EXP_2005F)
FOB_05['Ano'] = '2005'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_05.csv")
FOB_05.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Tabela final exportações 2005</span>

In [80]:
top=export_3(EXP_2015F)
result_05 = tab_final(top,FOB_15)
result_05

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,"Carnes de galos/galinhas, não cortadas em peda...","Enchidos e produtos semelhantes, de carne, de ...",Outros papéis e cartões dos tipos utilizados p...,6706901,2015
1,Albania,"Pedaços e miudezas, comestíveis de galos/galin...","Outros açúcares de cana, beterraba, sacarose q...","Outras carnes de suíno, congeladas",36546211,2015
2,Algeria,Outros açúcares de cana,"Milho em grão, exceto para semeadura","Óleo de soja, em bruto, mesmo degomado",992882604,2015
3,Americana Samoa,"Outros açúcares de cana, beterraba, sacarose q...","Outras máquinas e aparelhos para agricultura, ...",Partes de máquinas e aparelhos para agricultur...,40679,2015
4,Andorra,"Bulhão dourado (bullion doré), em formas bruta...",Joalheria de ouro do capitulo 71 da NCM,NaN,152825,2015
...,...,...,...,...,...,...
228,Virgin Islands (UK),"Aviões e outros veículos aéreos, a turbojato, ...","Outros quadros, etc, com aparelhos interruptor...","Outras carnes de suíno, congeladas",15561494,2015
229,Virgin Islands (USA),"Outros ladrilhos, etc, de cerâmica, vidrados, ...","Outras madeiras compensadas, constituídas excl...",Joalheria de ouro do capitulo 71 da NCM,177698,2015
230,Yemen,Outros açúcares de cana,"Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...",214301925,2015
231,Zambia,"Outros pneus novos, para veículos de construçã...",Semeadores-adubadores,Máquinas e aparelhos para indústria de panific...,5334118,2015


<span style="font-size: 35px;">Exportações 2004</span>

In [81]:
EXP_2004F=exp_final('EXP_2004.csv')

<span style="font-size: 20px;">Somatoria das exportações 2004</span>

In [82]:
FOB_04 = soma_exp(EXP_2004F)
FOB_04['Ano'] = '2004'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_04.csv")
FOB_04.to_csv(nome_arquivo_csv, index=False)
FOB_04

,Regiao,VL_FOB,Ano
210,United States,20002799431,2004
8,Argentina,7371118286,2004
40,China,5438637977,2004
141,Netherlands,4702038799,2004
74,Germany,4034398379,2004
...,...,...,...
135,Montserrat,7711,2004
82,Guernsey,7039,2004
174,San Marino,6514,2004
155,Palau,1422,2004


<span style="font-size: 35px;">Tabela final exportações 2004</span>

In [83]:
top=export_3(EXP_2004F)
result_04 = tab_final(top,FOB_04)
result_04

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,Obras de fios de ferro ou aço,"Caramelos, confeitos, dropes, pastilhas, e pro...","Outras preparações alimentícias com cacau, não...",172025,2004
1,Albania,"Outras carnes de suíno, congeladas","Outros açúcares de cana, beterraba, sacarose q...","Carnes desossadas de bovino, congeladas",18474087,2004
2,Algeria,"Açúcar de cana, em bruto","Carnes desossadas de bovino, congeladas","Trigo (exceto trigo duro ou para semeadura), e...",347610104,2004
3,Americana Samoa,"Preparações alimentícias e conservas, da espéc...",NaN,NaN,37955,2004
4,Andorra,"Ardósia, mesmo desbastada ou simplesmente cort...",Sutiãs e bustiês,"Calcinhas, de malha, de uso feminino, de fibra...",14510,2004
...,...,...,...,...,...,...
216,Virgin Islands (USA),"Outros ladrilhos, etc, de cerâmica, vidrados, ...",Outros móveis de madeira,"Outras partes de calçados, etc, de outras maté...",439576,2004
217,Yemen,"Carnes de galos/galinhas, não cortadas em peda...","Outros açúcares de cana, beterraba, sacarose q...","Trigo (exceto trigo duro ou para semeadura), e...",117792132,2004
218,Yugoslavia,"Outros açúcares de cana, beterraba, sacarose q...","Milho em grão, exceto para semeadura","Trigo (exceto trigo duro ou para semeadura), e...",27100644,2004
219,Zambia,"Fios de alumínio não ligado, com a maior dimen...",Outros tratores,"Outros bulldozers e angledozers, de lagartas",8456133,2004


In [84]:
#nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_04.csv")
#FOB_04.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Exportações 2003</span>

In [85]:
EXP_2003F= exp_final("EXP_2003.csv")
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\EXP_2003F.csv")
EXP_2003F.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 20px;">Somataria das exportações 2003</span>

In [86]:
FOB_03 = soma_exp(EXP_2003F)
FOB_03['Ano'] = '2003'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_03.csv")
FOB_03.to_csv(nome_arquivo_csv, index=False)
FOB_03

,Regiao,VL_FOB,Ano
209,United States,16659334047,2003
7,Argentina,4559972158,2003
39,China,4531201183,2003
139,Netherlands,4106474784,2003
74,Germany,3127899500,2003
...,...,...,...
212,Vanuatu,2351,2003
78,Greenland,2033,2003
40,Christmas Island,594,2003
44,Cook Islands,526,2003


<span style="font-size: 35px;">Tabela final exportações 2003</span>

In [87]:
top=export_3(EXP_2003F)
result_03 = tab_final(top,FOB_03)
result_03

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,"Outros leites, cremes de leite, concentrados, ...","Fio de raiom viscose, sem torção ou com torção...","Álcool etílico não desnaturado, com volume de ...",336773,2003
1,Albania,"Outras carnes de suíno, congeladas","Carnes de galos/galinhas, não cortadas em peda...","Carnes desossadas de bovino, congeladas",7801302,2003
2,Algeria,"Açúcar de cana, em bruto","Outros açúcares de cana, beterraba, sacarose q...","Carnes desossadas de bovino, congeladas",153704148,2003
3,Andorra,"Ardósia, mesmo desbastada ou simplesmente cort...",Sutiãs e bustiês,"Calcinhas, de malha, de uso feminino, de fibra...",40418,2003
4,Angola,"Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...","Outros leites, cremes de leite, concentrados, ...",234256437,2003
...,...,...,...,...,...,...
216,Virgin Islands (USA),"Outros ladrilhos, etc, de cerâmica, vidrados, ...","Outras partes de calçados, etc, de outras maté...","Carnes de bovinos, salgadas/em salmoura/secas/...",766994,2003
217,Yemen,"Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...","Tabaco não manufaturado, total ou parcialmente...",105384813,2003
218,Yugoslavia,"Outros açúcares de cana, beterraba, sacarose q...","Café não torrado, não descafeinado, em grão","Tabaco não manufaturado, total ou parcialmente...",12411892,2003
219,Zambia,Outros tratores,"Outros fios de alumínio, não ligados","Fios de alumínio não ligado, com a maior dimen...",5524835,2003


<span style="font-size: 35px;">Exportações 2002</span>

In [88]:
EXP_2002F= exp_final("EXP_2002.csv")

<span style="font-size: 25px;">Somatoria das exportações 2002</span>

In [89]:
FOB_02 = soma_exp(EXP_2002F)
FOB_02['Ano'] = '2002'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_02.csv")
FOB_02.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Tabela final exportações 2002</span>

In [90]:
top=export_3(EXP_2002F)
result_02 = tab_final(top,FOB_02)
result_02

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,"Fibras de raiom viscose, não cardadas, não pen...",NaN,NaN,1847,2002
1,Albania,"Outras carnes de suíno, congeladas","Carnes de galos/galinhas, não cortadas em peda...","Carnes desossadas de bovino, congeladas",4816026,2002
2,Algeria,"Outros açúcares de cana, beterraba, sacarose q...","Açúcar de cana, em bruto","Milho em grão, exceto para semeadura",86850505,2002
3,Andorra,"Ardósia, mesmo desbastada ou simplesmente cort...","Móveis de outras matérias, inclusive rotim, vi...",Estatuetas/outros objetos ornamentais de cerâm...,8372,2002
4,Angola,"Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...",Veículos automóveis de combate a incêndios,198313037,2002
...,...,...,...,...,...,...
208,Western Sahara,"Carnes desossadas de bovino, congeladas",NaN,NaN,68589,2002
209,Yemen,"Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...",Tratores de lagartas,102633570,2002
210,Yugoslavia,"Tabaco não manufaturado, total ou parcialmente...","Outros açúcares de cana, beterraba, sacarose q...","Outros grãos de soja, mesmo triturados",8603597,2002
211,Zambia,Outros tratores,"Aparelhos manuais para projetar, dispersar ou ...","Grades de discos, de uso agrícola, hortícola o...",1420365,2002


In [91]:
#nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_02.csv")
#FOB_02.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Exportações 2001</span>

In [92]:
EXP_2001F= exp_final("EXP_2001.csv")
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\EXP_2001F.csv")
EXP_2001F.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 20px;">Somataria das exportações 2001</span>

In [93]:
FOB_01 = soma_exp(EXP_2001F)
FOB_01['Ano'] = '2001'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_01.csv")
FOB_01.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Tabela final exportações 2001</span>

In [94]:
top=export_3(EXP_2001F)
result_01 = tab_final(top,FOB_01)
result_01

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,Partes superiores de calçados e seus componentes,Outras partes e acessórios para tratores e veí...,Rodas dentadas e outros órgãos elementares de ...,16652,2001
1,Albania,"Outras carnes de suíno, congeladas","Carnes de galos/galinhas, não cortadas em peda...","Carnes desossadas de bovino, congeladas",1584611,2001
2,Algeria,"Outros açúcares de cana, beterraba, sacarose q...","Açúcar de cana, em bruto",Cartuchos para espingardas ou carabinas de can...,44777751,2001
3,Andorra,"Caramelos, confeitos, dropes, pastilhas, e pro...","Charutos e cigarrilhas, que contenham tabaco",Joalheria de ouro do capitulo 71 da NCM,26895,2001
4,Angola,"Outros açúcares de cana, beterraba, sacarose q...",Veículos automóveis para transporte de dez pes...,"Carnes de galos/galinhas, não cortadas em peda...",141861333,2001
...,...,...,...,...,...,...
208,Virgin Islands (USA),Outras embarcações para o transporte de mercad...,"Outros ladrilhos, etc, de cerâmica, vidrados, ...","Pneumáticos novos, de borracha, dos tipos util...",982966,2001
209,Yemen,"Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...",Outras espingardas e carabinas de caça ou de t...,105250123,2001
210,Yugoslavia,"Milho em grão, exceto para semeadura","Bagaços e outros resíduos sólidos, da extração...",Outras misturas utilizadas como materia básica...,3360906,2001
211,Zambia,"Aparelhos manuais para projetar, dispersar ou ...",Outros tratores,Outros niveladores,1250490,2001


<span style="font-size: 35px;">Exportações 2000</span>

In [95]:
EXP_2000F= exp_final("EXP_2000.csv")

<span style="font-size: 25px;">Somatoria das exportações 2000</span>

In [96]:
FOB_00 = soma_exp(EXP_2000F)
FOB_00['Ano'] = '2000'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_00.csv")
FOB_00.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Tabela final exportações 2000</span>

In [97]:
top=export_3(EXP_2000F)
result_00 = tab_final(top,FOB_00)
result_00

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,"Outros couros e peles, de bovinos, ""wet blue""",NaN,NaN,8656,2000
1,Albania,"Outras carnes de suíno, congeladas","Pedaços e miudezas, comestíveis de galos/galin...","Carnes desossadas de bovino, congeladas",228569,2000
2,Algeria,"Outros açúcares de cana, beterraba, sacarose q...","Outros tubos de ferro/aço, sem costura, para o...","Açúcar de cana, em bruto",41405084,2000
3,Andorra,Joalheria de ouro do capitulo 71 da NCM,NaN,NaN,5638,2000
4,Angola,"Carnes de galos/galinhas, não cortadas em peda...","Outros açúcares de cana, beterraba, sacarose q...","Pontes e elementos de pontes, de ferro fundido...",105978131,2000
...,...,...,...,...,...,...
206,Western Sahara,Preparações para dar brilho a pinturas de carr...,NaN,NaN,29160,2000
207,Yemen,"Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...","Tabaco não manufaturado, total ou parcialmente...",75141703,2000
208,Yugoslavia,"Café não torrado, não descafeinado, em grão",Outras misturas utilizadas como materia básica...,Tecidos de algodão que contenham pelo menos 85...,677130,2000
209,Zambia,Chassis com motor para veículos automóveis tra...,Outros aparelhos para pulverizar fungicidas/in...,"Grades de discos, de uso agrícola, hortícola o...",692961,2000


In [98]:
#nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_00.csv")
#FOB_00.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Exportações 1999</span>

In [99]:
EXP_1999F= exp_final("EXP_1999.csv")
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\EXP_1999F.csv")
EXP_1999F.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 25px;">Somataria das exportações 1999</span>

In [100]:
FOB_99= soma_exp(EXP_1999F)
FOB_99['Ano'] = '1999'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_99.csv")
FOB_99.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Tabela final exportações 1999</span>

In [101]:
top=export_3(EXP_1999F)
result_99 = tab_final(top,FOB_99)
result_99

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,Outros artigos do capitulo 71 da NCM,"Outros objetos de vidro, para serviço de mesa/...",NaN,89027,1999
1,Albania,"Outras carnes de suíno, congeladas","Carnes de galos/galinhas, não cortadas em peda...","Pedaços e miudezas, comestíveis de galos/galin...",419310,1999
2,Algeria,"Outros açúcares de cana, beterraba, sacarose q...","Outros tubos de ferro/aço, sem costura, para o...","Açúcar de cana, em bruto",64004397,1999
3,Andorra,Joalheria de ouro do capitulo 71 da NCM,"Outros tecidos de filamentos sintéticos, de fi...","Outros tecidos, que contenham menos de 85 %, e...",5684,1999
4,Angola,"Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...","Carnes de bovinos, salgadas/em salmoura/secas/...",63941173,1999
...,...,...,...,...,...,...
206,Virgin Islands (USA),Bauxita não calcinada (minério de alumínio),Recipientes para gases comprimidos ou liquefei...,"Preparações alimentícias e conservas, da espéc...",18943300,1999
207,Yemen,"Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...","Açúcar de cana, em bruto",58349396,1999
208,Yugoslavia,"Outros açúcares de cana, beterraba, sacarose q...","Café não torrado, não descafeinado, em grão","Bagaços e outros resíduos sólidos, da extração...",1839727,1999
209,Zambia,Outros niveladores,"Aparelhos manuais para projetar, dispersar ou ...",Outros aparelhos para pulverizar fungicidas/in...,590255,1999


<span style="font-size: 35px;">Exportações 1998</span>

In [102]:
EXP_1998F= exp_final("EXP_1998.csv")

<span style="font-size: 25px;">Somatoria das exportações 1998</span>

In [103]:
FOB_98 = soma_exp(EXP_1998F)
FOB_98['Ano'] = '1998'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_98.csv")
FOB_98.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Tabela final exportações 1998</span>

In [104]:
top=export_3(EXP_1998F)
result_98 = tab_final(top,FOB_98)
result_98

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,"Outros objetos de vidro, para serviço de mesa/...",NaN,NaN,30366,1998
1,Albania,"Outras carnes de suíno, congeladas",Outras bobinas de reatância e de auto-indução,"Pedaços e miudezas, comestíveis de galos/galin...",492106,1998
2,Algeria,"Outros açúcares de cana, beterraba, sacarose q...","Açúcar de cana, em bruto","Outros papéis/cartões, fibra obtida por proces...",37417782,1998
3,Americana Samoa,"Calças, jardineiras, bermudas e shorts (calçõe...","Camisetas, incluindo as interiores, de malha, ...",NaN,400,1998
4,Andorra,Joalheria de ouro do capitulo 71 da NCM,Pedras lapidadas/trabalhadas do capítulo 71 da...,NaN,1214,1998
...,...,...,...,...,...,...
204,Wake Island,"Bagaços e outros resíduos sólidos, da extração...",NaN,NaN,170000,1998
205,Yemen,"Outros açúcares de cana, beterraba, sacarose q...","Açúcar de cana, em bruto","Outros papéis/cartões, fibra obtida por proces...",30569350,1998
206,Yugoslavia,"Café não torrado, não descafeinado, em grão","Bagaços e outros resíduos sólidos, da extração...",Outras misturas utilizadas como materia básica...,4247757,1998
207,Zambia,"Aparelhos manuais para projetar, dispersar ou ...",Secadores para produtos agrícolas,"Tijolos ou placas refratárias, contendo, em pe...",733977,1998


In [105]:
#nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_98.csv")
#FOB_98.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 35px;">Exportações 1997</span>

In [106]:
EXP_1997F= exp_final("EXP_1997.csv")
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\EXP_1997F.csv")
EXP_1997F.to_csv(nome_arquivo_csv, index=False)

<span style="font-size: 25px;">Somataria das exportações 1997</span>

In [107]:
FOB_97= soma_exp(EXP_1997F)
FOB_97['Ano'] = '1997'
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\FOB_97.csv")
FOB_97.to_csv(nome_arquivo_csv, index=False)
FOB_97

,Regiao,VL_FOB,Ano
197,United States,9268838126,1997
6,Argentina,6765349920,1997
128,Netherlands,3997768168,1997
92,Japan,3065771924,1997
68,Germany,2607749915,1997
...,...,...,...
125,Myanmar,3822,1997
135,Norfolk Island,1374,1997
111,Maldives,995,1997
187,Tonga,500,1997


<span style="font-size: 35px;">Tabela final exportações 1997</span>

In [108]:
top=export_3(EXP_1997F)
result_97 = tab_final(top,FOB_97)
result_97

,Regiao,top1,top2,top3,VL_FOB,Ano
0,Afghanistan,Chá verde (não fermentado) em embalagens imedi...,"Outros objetos de vidro, para serviço de mesa/...","Automóveis com motor explosão, 1500 < cm3 <=30...",143189,1997
1,Albania,"Café não torrado, não descafeinado, em grão",Torrefadores,"Carnes desossadas de bovino, congeladas",401206,1997
2,Algeria,"Outros açúcares de cana, beterraba, sacarose q...","Açúcar de cana, em bruto","Outros papéis/cartões, fibra obtida por proces...",36388064,1997
3,Angola,"Outros açúcares de cana, beterraba, sacarose q...","Carnes de galos/galinhas, não cortadas em peda...","Açúcar de cana, em bruto",81210109,1997
4,Anguilla,"Outros ladrilhos, etc, de cerâmica, vidrados, ...","Outras carnes de suíno, congeladas","Madeira compensada com folhas <=6 mm, face de ...",136694,1997
...,...,...,...,...,...,...
204,Virgin Islands (USA),Bauxita não calcinada (minério de alumínio),"Pneumáticos novos, de borracha, dos tipos util...","Outros veículos automóveis com motor diesel, p...",1178530,1997
205,Yemen,"Outros açúcares de cana, beterraba, sacarose q...","Açúcar de cana, em bruto","Outros papéis/cartões, fibra obtida por proces...",105898014,1997
206,Yugoslavia,"Minérios de ferro e seus concentrados, exceto ...",Outros minérios de manganês e seus concentrado...,"Bagaços e outros resíduos sólidos, da extração...",6663090,1997
207,Zambia,Veículos automóveis para transporte de dez pes...,"Tijolos ou placas refratárias, contendo, em pe...","Aparelhos manuais para projetar, dispersar ou ...",1509648,1997


In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_97.csv")
result_97.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_98.csv")
result_98.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_99.csv")
result_99.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_00.csv")
result_00.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_01.csv")
result_01.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_02.csv")
result_02.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_03.csv")
result_03.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_04.csv")
result_04.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_05.csv")
result_05.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_06.csv")
result_06.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_07.csv")
result_07.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_08.csv")
result_08.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_09.csv")
result_09.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_10.csv")
result_10.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_11.csv")
result_11.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_12.csv")
result_12.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_13.csv")
result_13.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_14.csv")
result_14.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_15.csv")
result_15.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_16.csv")
result_16.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_17.csv")
result_17.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_18.csv")
result_18.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_19.csv")
result_19.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_20.csv")
result_20.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_21.csv")
result_21.to_csv(nome_arquivo_csv, index=False)

In [ ]:
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\result_22.csv")
result_22.to_csv(nome_arquivo_csv, index=False)

In [109]:
gustavo = pd.concat([FOB_97, FOB_98,FOB_99,FOB_00,FOB_01,FOB_02,FOB_03,FOB_04,FOB_05,FOB_06,FOB_07,FOB_08,FOB_09,FOB_10
                ,FOB_11,FOB_12,FOB_13,FOB_14,FOB_15,FOB_16,FOB_17,FOB_18,FOB_19,FOB_20,FOB_21,FOB_22])
gustavo

,Regiao,VL_FOB,Ano
197,United States,9268838126,1997
6,Argentina,6765349920,1997
128,Netherlands,3997768168,1997
92,Japan,3065771924,1997
68,Germany,2607749915,1997
...,...,...,...
92,Guernsey,1783,2022
31,Bouvet Island,218,2022
161,Northern Marianas,180,2022
33,British Indian Ocean Territory,175,2022


In [123]:
gustavo['Regiao'] = gustavo['Regiao'].replace('United States of America', 'United States')
gustavo['Regiao'] = gustavo['Regiao'].replace('united states of America', 'United States')
gustavo['Regiao'] = gustavo['Regiao'].replace('French Guyana', 'French Guiana')
gustavo['Regiao'] = gustavo['Regiao'].replace('Serbia', 'Republic of Serbia')
nome_arquivo_csv = (r"C:\Users\lucas\Downloads\gustavo.csv")
gustavo.to_csv(nome_arquivo_csv, index=False)


In [120]:
gustavo['Regiao'].value_counts()

Regiao
United States                                   26
Suriname                                        26
Cape Verde                                      26
Ethiopia                                        26
Moldova                                         26
                                                ..
Not declared                                     1
Central Banks                                    1
Czechoslovakia                                   1
\r\nHeard Island and McDonald Islands            1
South Georgia and the South Sandwich Islands     1
Name: count, Length: 263, dtype: int64